In [1]:
%reset -f

In [2]:
from sssom.io import parse_sssom_table
import pandas as pd

fn = "demo.tsv"

### Annotate

In [3]:
!runoak -i sqlite:obo:mondo annotate "Marfan syndrome" --matches-whole-text

object_id: MONDO:0007947
object_label: Marfan syndrome
matches_whole_text: true
subject_start: 1
subject_end: 15
subject_label: Marfan syndrome

---
object_id: MONDO:0007947
object_label: Marfan syndrome
matches_whole_text: true
subject_start: 1
subject_end: 15
subject_label: Marfan syndrome


### Lexmatch

In [4]:
%%time
!runoak -i sqlite:obo:mondo -a sqlite:obo:doid lexmatch i^MONDO: @  i^DOID: -o {fn}
# !head demo.tsv
msdf = parse_sssom_table(fn)
msdf.df.head()

CPU times: user 48.1 s, sys: 1.01 s, total: 49.1 s
Wall time: 4min 41s


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0000001,disease or disorder,skos:closeMatch,DOID:4,disease,semapv:LexicalMatching,oaklib,0.5,oio:hasDbXref,oio:hasDbXref,mesh:d004194
1,MONDO:0000001,disease or disorder,skos:closeMatch,DOID:4,disease,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,rdfs:label,disease
2,MONDO:0000004,adrenocortical insufficiency,skos:closeMatch,DOID:10493,adrenal cortical hypofunction,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,oio:hasExactSynonym,adrenal cortical insufficiency
3,MONDO:0000004,adrenocortical insufficiency,skos:closeMatch,DOID:10493,adrenal cortical hypofunction,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,oio:hasExactSynonym,corticoadrenal insufficiency
4,MONDO:0000004,adrenocortical insufficiency,skos:closeMatch,DOID:10493,adrenal cortical hypofunction,semapv:LexicalMatching,oaklib,0.5,oio:hasExactSynonym,rdfs:label,adrenal cortical hypofunction


### Relationships

In [5]:
!runoak -i sqlite:obo:mondo relationships  "Marfan syndrome" -o {fn}
# !head demo.tsv
df = pd.read_csv(fn, sep="\t")
df.head()

,subject,subject_label,predicate,predicate_label,object,object_label
0,MONDO:0007947,Marfan syndrome,RO:0004003,has material basis in germline mutation in,<http://identifiers.org/hgnc/3603>,FBN1
1,MONDO:0007947,Marfan syndrome,rdfs:subClassOf,NaN,MONDO:0000426,autosomal dominant disease
2,MONDO:0007947,Marfan syndrome,rdfs:subClassOf,NaN,MONDO:0002254,syndromic disease
3,MONDO:0007947,Marfan syndrome,rdfs:subClassOf,NaN,MONDO:0017310,Marfan and Marfan-related disorder
4,MONDO:0007947,Marfan syndrome,rdfs:subClassOf,NaN,MONDO:0800091,overgrowth or tall stature syndrome with skele...


In [6]:
# RO:0004003 => "has material basis in germline mutation in"
!runoak -i sqlite:obo:mondo relationships  --predicates RO:0004003 -o {fn}
# !head demo.tsv
df = pd.read_csv(fn, sep="\t")
df.head()

,subject,subject_label,predicate,predicate_label,object,object_label
0,MONDO:0000208,"microcephaly, short stature, and impaired gluc...",RO:0004003,has material basis in germline mutation in,<http://identifiers.org/hgnc/28403>,TRMT10A
1,MONDO:0000510,synucleinopathy,RO:0004003,has material basis in germline mutation in,<http://identifiers.org/hgnc/11138>,SNCA
2,MONDO:0000764,epithelial-stromal TGFBI dystrophy,RO:0004003,has material basis in germline mutation in,<http://identifiers.org/hgnc/11771>,TGFBI
3,MONDO:0000902,agenesis of the corpus callosum with periphera...,RO:0004003,has material basis in germline mutation in,<http://identifiers.org/hgnc/10914>,SLC12A6
4,MONDO:0000908,arrhythmogenic right ventricular dysplasia 13,RO:0004003,has material basis in germline mutation in,<http://identifiers.org/hgnc/2511>,CTNNA3


### Fill table

In [26]:
import numpy as np
# Create an incomplete table
table = {
        'id': ["MONDO:0007947", "MONDO:0000510"],
        'label': [np.NaN, np.NaN],
        'definition': [np.NaN, np.NaN],
        'mappings': [np.NaN, np.NaN],
        'ancestors': [np.NaN, np.NaN]
        }

data = pd.DataFrame(data=table)
data.to_csv(fn, sep="\t", index=False)
data

,id,label,definition,mappings,ancestors
0,MONDO:0007947,NaN,NaN,NaN,NaN
1,MONDO:0000510,NaN,NaN,NaN,NaN


In [27]:
# Fill it through OAK
!runoak -i sqlite:obo:mondo fill-table {fn} -o {fn}
df = pd.read_csv(fn, sep="\t")
df.head()

,id,label,definition,mappings,ancestors
0,MONDO:0007947,Marfan syndrome,A disorder of the connective tissue. Connectiv...,DOID:14323|GARD:0006975|ICD9:759.82|MESH:D0083...,MONDO:0007947|BFO:0000001|BFO:0000002|BFO:0000...
1,MONDO:0000510,synucleinopathy,A neurodegenerative disease that is characteri...,DOID:0050890|MESH:D000080874|UMLS:C5191670|DOI...,MONDO:0000510|BFO:0000001|BFO:0000002|BFO:0000...
